In [34]:
# Importing necessary models
import warnings
warnings.filterwarnings('ignore')

import smtplib
import pandas as pd
import numpy as np
import datetime as dt

import os
from sqlalchemy import *
from sqlalchemy import create_engine

main_dir = os.getcwd()

dbs_dir = 'C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Data\\DBs'

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\DataCollection')

from alphavantage import *
from yahoo_query import *

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\Options')
from optionsFunctions import *

vix_options_dir = 'D:\\Options Data\\VIX'
spx_options_dir = 'D:\\Options Data\\SPX'
spx_intraday_dir = 'D:\\Options Data\\SPX Intraday'
cleaned_dir = 'D:\\Options Data\\Monthly DBs'
#'vix_2018-07-30

In [16]:
def monthly_spx_intraday_prices(curr_date):
    spx_dir = 'D:\\Options Data\\SPX Intraday'
    os.chdir(spx_dir)
    spx_intraday_dfs = []
    curr_spx_price_files = list(filter(lambda x: curr_date.strftime("%Y-%m") in x, os.listdir('.')))
    for filename in curr_spx_price_files:
        if filename.endswith(".csv"):
            curr_df = pd.read_csv(filename, index_col = 0).dropna()
            curr_df.index = pd.to_datetime(curr_df.index)
            curr_df = curr_df[curr_df.index.time >= dt.time(9,30)]
            spx_intraday_dfs.append(curr_df)
        else:
            continue
    return pd.concat(spx_intraday_dfs)

def monthly_spx_options(curr_date):
    spx_options_dir = 'D:\\Options Data\\SPX'
    os.chdir(spx_options_dir)
    spx_calls_dfs = []
    spx_puts_dfs = []
    curr_spx_options_files = list(filter(lambda x: curr_date.strftime("%Y-%m") in x, os.listdir('.')))
    keep_columns = ['Strike','Expiry','Type','Symbol','Last','Bid','Ask','IV','Underlying_Price']
    for filename in curr_spx_options_files:
        if filename.endswith(".csv"):
            curr_options = pd.read_csv(filename, index_col = 0).set_index('Quote_Time')[keep_columns]
            curr_options.index = pd.to_datetime(curr_options.index) - dt.timedelta(hours = 4)
            curr_options.index = curr_options.index.map(lambda x: x.replace(second=0))
            curr_options['Expiry'] = pd.to_datetime(curr_options.Expiry)
            curr_options['DTE'] = (curr_options.Expiry - curr_options.index).dt.days + 1
            curr_options['Mid'] = (curr_options['Ask'] + curr_options['Bid'])/2
            greeks = all_greeks(curr_options, interest_rate = 0.023869, q = 0, year = 252)
            greeks.index = pd.to_datetime(curr_options.index)
            puts = greeks[greeks.Type == 'put']
            calls = greeks[greeks.Type == 'call']
            spx_calls_dfs.append(calls)
            spx_puts_dfs.append(puts)
        else:
            continue
    puts = pd.concat(spx_puts_dfs)
    calls = pd.concat(spx_calls_dfs)
    return puts, calls

In [3]:
date_range = [dt.datetime(2019, 1, 1)]

for curr_date in date_range:
    os.chdir(cleaned_dir)
    latest_month = curr_date.month
    db_name = 'sqlite:///intraday_{0}-{1}.db'.format(curr_date.year,latest_month)
    
    options_engine = create_engine(db_name, echo=False)
    
    spx_intraday = monthly_spx_intraday_prices(curr_date)
    puts, calls = monthly_spx_options(curr_date)
    
    puts.to_sql('spx_puts', con=options_engine, if_exists='append', index_label = 'quoteTime')
    calls.to_sql('spx_calls', con=options_engine, if_exists='append', index_label = 'quoteTime')
    spx_intraday.to_sql('spx_prices', con=options_engine, if_exists='append')
    print(curr_date)

2019-01-01 00:00:00


In [4]:
spx_ticker = '^GSPC'
spx = yahoo_query(spx_ticker, dt.datetime(2018,1,1))
spx.hist_prices_query()
spx = spx.hist_prices 
spx.columns = [col.replace('^', '') for col in spx.columns.tolist()]

vix_ticker = '^VIX'
vix = yahoo_query(vix_ticker, dt.datetime(2018,1,1))
vix.hist_prices_query()
vix = vix.hist_prices 
vix.columns = [col.replace('^', '') for col in vix.columns.tolist()]

df = spx[['GSPC_open','GSPC_close']].join(vix[['VIX_open']])
df['LB'] = df['GSPC_open']*(1 - df['VIX_open']/(100*np.sqrt(252)))
df['UB'] = df['GSPC_open']*(1 + df['VIX_open']/(100*np.sqrt(252)))

In [83]:
spx_options_dir = 'D:\\Options Data\\SPX'
os.chdir(spx_options_dir)
spx_calls_dfs = []
spx_puts_dfs = []
curr_spx_options_files = os.listdir('.')


keep_columns = ['Strike','Expiry','Type','Symbol','Last','Bid','Ask','IV','Underlying_Price']

dates = []

bod_put_credits = []
bod_call_credits = []

eod_put_credits = []
eod_call_credits = []

for curr_date, row in df.iterrows():
    curr_date_string = curr_date.strftime("%Y-%m-%d")
    curr_filename_bod = 'spx_{}-06-35.csv'.format(curr_date_string)
    curr_filename_eod = 'spx_{}-13-00.csv'.format(curr_date_string)
    
    try:
        curr_options_bod = pd.read_csv(curr_filename_bod, index_col = 0).set_index('Quote_Time')[keep_columns]
        curr_options_bod.index = pd.to_datetime(curr_options_bod.index) - dt.timedelta(hours = 4)
        curr_options_bod.index = curr_options_bod.index.map(lambda x: x.replace(second=0))
        curr_options_bod['Expiry'] = pd.to_datetime(curr_options_bod.Expiry)
        curr_options_bod['DTE'] = (curr_options_bod.Expiry - curr_options_bod.index).dt.days + 1
        curr_options_bod['Mid'] = (curr_options_bod['Ask'] + curr_options_bod['Bid'])/2

        curr_options_bod = curr_options_bod[curr_options_bod.DTE == curr_options_bod.sort_values('DTE').iloc[0,9]]

        put_spread_bod = curr_options_bod[(curr_options_bod.Type == 'put') & 
                                      (curr_options_bod.Strike <= row.LB)].sort_values('Strike').tail(2)
        call_spread_bod = curr_options_bod[(curr_options_bod.Type == 'call') & 
                                       (curr_options_bod.Strike >= row.UB)].sort_values('Strike').head(2)

        put_credit_bod = put_spread_bod.Mid.diff().values[-1]
        call_credit_bod = abs(call_spread_bod.Mid.diff()).values[-1]

        curr_options_eod = pd.read_csv(curr_filename_eod, index_col = 0).set_index('Quote_Time')[keep_columns]
        curr_options_eod.index = pd.to_datetime(curr_options_eod.index) - dt.timedelta(hours = 4)
        curr_options_eod.index = curr_options_eod.index.map(lambda x: x.replace(second=0))
        curr_options_eod['Expiry'] = pd.to_datetime(curr_options_eod.Expiry)
        curr_options_eod['DTE'] = (curr_options_eod.Expiry - curr_options_eod.index).dt.days + 1
        curr_options_eod['Mid'] = (curr_options_eod['Ask'] + curr_options_eod['Bid'])/2

        put_credit_eod = curr_options_eod[curr_options_eod.Symbol.isin(put_spread_bod.Symbol.tolist())].iloc[1,10] - curr_options_eod[curr_options_eod.Symbol.isin(put_spread_bod.Symbol.tolist())].iloc[0,10]
        call_credit_eod = curr_options_eod[curr_options_eod.Symbol.isin(call_spread_bod.Symbol.tolist())].iloc[0,10] - curr_options_eod[curr_options_eod.Symbol.isin(call_spread_bod.Symbol.tolist())].iloc[1,10]

        dates.append(curr_date)
        bod_put_credits.append(put_credit_bod)
        bod_call_credits.append(call_credit_bod)
        eod_put_credits.append(put_credit_eod)
        eod_call_credits.append(call_credit_eod)
    except:
        continue

In [84]:
pd.DataFrame({'bod_puts':bod_put_credits,
              'bod_calls':bod_call_credits,
              'eod_puts':eod_put_credits,
              'eod_calls':eod_call_credits}, index = dates).to_csv('spreads.csv')

In [10]:
n = norm.pdf
N = norm.cdf

def bs_price(cp_flag,S,K,T,r,v,q=0.0):
    d1 = (np.log(S/K)+(r+v*v/2.)*T)/(v*np.sqrt(T))
    d2 = d1-v*np.sqrt(T)
    if cp_flag == 'c':
        price = S*np.exp(-q*T)*N(d1)-K*np.exp(-r*T)*N(d2)
    else:
        price = K*np.exp(-r*T)*N(-d2)-S*np.exp(-q*T)*N(-d1)
    return price

def bs_vega(cp_flag,S,K,T,r,v,q=0.0):
    d1 = (np.log(S/K)+(r+v*v/2.)*T)/(v*np.sqrt(T))
    return S * np.sqrt(T)*n(d1)

def find_vol(target_value, call_put, S, K, T, r):
    MAX_ITERATIONS = 100
    PRECISION = 1.0e-5

    sigma = 0.5
    for i in range(0, MAX_ITERATIONS):
        price = bs_price(call_put, S, K, T, r, sigma)
        vega = bs_vega(call_put, S, K, T, r, sigma)

        price = price
        diff = target_value - price  # our root

        print( i, sigma, diff)

        if (abs(diff) < PRECISION):
            return sigma
        sigma = sigma + diff/vega # f(x) / f'(x)

    # value wasn't found, return best guess so far
    return sigma

In [14]:
find_vol(3, 'c', 2664.34, 2665, 0.038888889, 0.00000604)

0 0.5 -101.446281803
1 0.0154940910886 0.0709204380282
2 0.0158335052139 -4.9839602525e-06


0.015833505213926888

In [123]:
os.chdir(cleaned_dir)
vix_intraday = pd.read_csv('vix_intraday.csv', index_col = 0)

vix_intraday.columns = ['Quote_Time', 'VIX', 'Date']
del vix_intraday['Date']
vix_intraday.Quote_Time = pd.to_datetime(vix_intraday.Quote_Time)
vix_intraday = vix_intraday.sort_values('Quote_Time')

curr_day = dt.datetime(2018,7,30)
while curr_day < dt.datetime.today():
    
    curr_date_string = curr_day.strftime("%Y-%m-%d")
    
    os.chdir(spx_options_dir)
    spx_options_filenames = list(filter(lambda x: curr_date_string in x, os.listdir('.')))
    if spx_options_filenames != []:
        print(curr_date_string)
    break
    curr_day = curr_day + dt.timedelta(days = 1)

2018-07-30


In [124]:
curr_date_string = curr_day.strftime("%Y-%m-%d")
os.chdir(spx_options_dir)
spx_options_filenames = list(filter(lambda x: curr_date_string in x, os.listdir('.')))

spx_options = []

if spx_options_filenames != []:
    for file in spx_options_filenames:
        curr_options = pd.read_csv(file, index_col = 0)
        curr_options.Expiry = pd.to_datetime(curr_options.Expiry)
        curr_options.Quote_Time = pd.to_datetime(curr_options.Quote_Time).map(lambda x: x.replace(second=0))
        
        if file.split('.')[0][-5:] == '06-30':
            quote_open_time = curr_options.Quote_Time[0]
            time_adj = quote_open_time - dt.datetime(quote_open_time.year, quote_open_time.month, quote_open_time.day, 9, 30)
        
        curr_options.Quote_Time = curr_options.Quote_Time - time_adj
        
        curr_options['DTE'] = (curr_options.Expiry - curr_options.Quote_Time).dt.days + 1
        
        curr_options.Expiry = curr_options.Expiry + dt.timedelta(seconds = 60*60*16 + 1)
        curr_options = curr_options[curr_options['DTE'] <= 1].reset_index(drop = True)
        curr_options['TradeDate'] = curr_options.Quote_Time.dt.date
        
        del curr_options['Chg'], curr_options['PctChg'], curr_options['Last_Trade_Date']
        if len(curr_options) != 0:
            spx_options.append(curr_options)
            
spx_options = pd.concat(spx_options, axis = 0).reset_index(drop = True)
keep_contracts = spx_options[['TradeDate','Symbol']].groupby('Symbol').count().sort_values('TradeDate')
spx_options = spx_options[spx_options.Symbol.isin(keep_contracts[keep_contracts.TradeDate >= 79].index)].reset_index(drop = True)
spx_options = spx_options.sort_values('Quote_Time').reset_index(drop = True)

Index(['Strike', 'Expiry', 'Type', 'Symbol', 'Last', 'Bid', 'Ask', 'Vol',
       'Open_Int', 'IV', 'Root', 'Underlying_Price', 'Quote_Time', 'DTE',
       'TradeDate'],
      dtype='object')